# A 3D Rosenbrock example for GPModel class

In [1]:
import torch
import logging
import os
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood
from lume_model.models import TorchModel
from lume_model.variables import ScalarVariable, DistributionVariable
from lume_model.models.gp_model import GPModel

## Create and train a GP

In [2]:
# Define the 3D Rosenbrock function
def rosenbrock(X):
    x1, x2, x3 = X[..., 0], X[..., 1], X[..., 2]
    return (1 - x1)**2 + 100 * (x2 - x1**2)**2 + (1 - x2)**2 + 100 * (x3 - x2**2)**2

In [3]:
# Generate training data
train_x = torch.rand(20, 3) * 4 - 2  # 20 points in 3D space, scaled to [-2, 2]
train_y = rosenbrock(train_x).unsqueeze(-1)  # Compute the Rosenbrock function values

# Define the GP model
gp_model = SingleTaskGP(train_x.to(dtype=torch.double), train_y.to(dtype=torch.double))

# Fit the model
mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
fit_gpytorch_model(mll)

# Evaluate the model on test data
test_x = torch.rand(10, 3) * 4 - 2  # 10 new points in 3D space
gp_model.eval()
posterior = gp_model.posterior(test_x)

# Get the mean and variance of the posterior
mean = posterior.mean
variance = posterior.variance

print("Posterior mean: ", mean)
print("Posterior variance: ", variance)

/opt/miniconda3/envs/lume-latest-310/lib/python3.10/site-packages/botorch/models/utils/assorted.py:174: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/opt/miniconda3/envs/lume-latest-310/lib/python3.10/site-packages/botorch/models/utils/assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([868.5472], dtype=torch.float64), std = tensor([883.1459], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


Posterior mean:  tensor([[744.3052],
        [744.5992],
        [744.9520],
        [744.3029],
        [744.0606],
        [744.2716],
        [744.2654],
        [744.3010],
        [744.3029],
        [744.3132]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Posterior variance:  tensor([[74.7112],
        [74.7111],
        [74.7099],
        [74.7112],
        [74.7099],
        [74.7112],
        [74.7112],
        [74.7112],
        [74.7112],
        [74.7112]], dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)


## LUME-Model import

In [4]:
# Define input variables
input_variables = [ScalarVariable(name="x1"), ScalarVariable(name="x2"), ScalarVariable(name="x3")]

# Define output variables
# Currently the "distribution_type" field doesn't do anything
output_variables = [DistributionVariable(name="output1", distribution_type="MultiVariateNormal")]

# Create lume_model instance
gp_lume_model = GPModel(model=gp_model, input_variables=input_variables, output_variables=output_variables)

#### Evaluate model and run methods

In [5]:
input_x = torch.rand(3, 3) * 4 - 2  # 3 new points in 3D space
input_dict = {"x1": input_x[:,0].to(dtype=torch.double),
              "x2": input_x[:,1].to(dtype=torch.double),
              "x3": input_x[:,2].to(dtype=torch.double)
             }

In [6]:
# Evaluate function returns a torch.distributions.Distribution
lume_dist = gp_lume_model.evaluate(input_dict)["output1"]

In [7]:
torch.manual_seed(0);
rand_test = torch.rand(1, 3)

In [8]:
print("Posterior mean:", lume_dist.mean)
print("Posterior Variance ", lume_dist.variance)
print("Log Likelihood", lume_dist.log_prob(rand_test))
print("Rsample ", lume_dist.rsample(torch.Size([3])))

Posterior mean: tensor([744.3032, 744.4946, 744.5623], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
Posterior Variance  tensor([74.7112, 74.7111, 74.7073], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Log Likelihood tensor([-11123.0036], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[747.4524, 747.1375, 749.7789],
        [745.4649, 746.3096, 751.5718],
        [745.7226, 752.3225, 744.1726]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)


### Outputs with original model

In [9]:
posterior = gp_model.posterior(input_x)
botorch_dist = posterior.distribution

In [10]:
print("Posterior mean:", botorch_dist.mean)
print("Posterior Variance ", botorch_dist.variance)
print("Log Likelihood", botorch_dist.log_prob(rand_test))
print("Rsample ", botorch_dist.rsample(torch.Size([3])))

Posterior mean: tensor([744.3032, 744.4946, 744.5623], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
Posterior Variance  tensor([74.7112, 74.7111, 74.7073], dtype=torch.float64,
       grad_fn=<ExpandBackward0>)
Log Likelihood tensor([-11123.0036], dtype=torch.float64, grad_fn=<SubBackward0>)
Rsample  tensor([[751.9069, 738.2463, 745.9490],
        [753.3635, 769.2700, 744.3432],
        [743.9186, 739.6066, 753.8443]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)
